## Full Pipeline: BERTIN Binary Classification on Translated Spanish Tweets

In [ ]:
!pip install -U transformers datasets scikit-learn optuna wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: wandb
    Found existing installation: wandb 0.20.1
    Uninstalling wandb-0.20.1:
      Successfully uninstalled wandb-0.20.1
  Attempting uninstall: datase

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Load dataset as a DataFrame
df = pd.read_csv('/content/drive/MyDrive/traducido_dataset_unificado_v_30jun.csv')

# Preview the dataset
df

/tmp/ipython-input-8-3159178268.py:4: DtypeWarning: Columns (9,10,11,12,13,19,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/traducido_dataset_unificado_v_30jun.csv')


,source_dataset,target_column,subcategory_combined,text,text_english,text_spanish,id,subcategory_general,subcategory_specific,rewire_id,...,TR,AG,label,test_case,source,language,task1,task2,detected_language,multiclass_final
0,edos_labelled_aggregated,0,NaN,"In Nigeria, if you rape a woman, the men rape ...","In Nigeria, if you rape a woman, the men rape ...","En Nigeria, si violas a una mujer, ¡los hombre...",NaN,NaN,NaN,sexism2022_english-9609,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN
1,edos_labelled_aggregated,0,NaN,"Then, she's a keeper. 😉","Then, she's a keeper. 😉","Entonces, ella es una guardiana. 😉",NaN,NaN,NaN,sexism2022_english-16993,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN
2,edos_labelled_aggregated,0,NaN,This is like the Metallica video where the poo...,This is like the Metallica video where the poo...,Esto es como el vídeo de Metallica donde el po...,NaN,NaN,NaN,sexism2022_english-13149,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN
3,edos_labelled_aggregated,0,NaN,woman?,woman?,¿mujer?,NaN,NaN,NaN,sexism2022_english-13021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,so,NaN
4,edos_labelled_aggregated,0,NaN,I bet she wished she had a gun,I bet she wished she had a gun,Apuesto a que deseaba tener un arma.,NaN,NaN,NaN,sexism2022_english-966,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44347,EXIST2021_training,0,NaN,"Estamos igual sin pareja, pero puedes besar a ...","We're the same without a partner, but you can ...","Estamos igual sin pareja, pero puedes besar a ...",6973.0,NaN,NaN,NaN,...,NaN,NaN,NaN,EXIST2021,twitter,es,non-sexist,non-sexist,es,NaN
44348,EXIST2021_training,0,NaN,2020 hijo de re mil putas,2020 son of a bitch,2020 hijo de re mil putas,6974.0,NaN,NaN,NaN,...,NaN,NaN,NaN,EXIST2021,twitter,es,non-sexist,non-sexist,es,NaN
44349,EXIST2021_training,0,NaN,SEGURAMENTE ESTA CHICA NO COBRA EL DINERO QUE ...,SURELY THIS GIRL DOESN'T GET PAID THE MONEY SH...,SEGURAMENTE ESTA CHICA NO COBRA EL DINERO QUE ...,6975.0,NaN,NaN,NaN,...,NaN,NaN,NaN,EXIST2021,twitter,es,non-sexist,non-sexist,pt,NaN
44350,EXIST2021_training,1,NaN,@safetyaitana mi madre dice q va fea y i agree,@safetyaitana my mother says she looks ugly an...,@safetyaitana mi madre dice q va fea y estoy d...,6976.0,NaN,NaN,NaN,...,NaN,NaN,NaN,EXIST2021,twitter,es,sexist,objectification,ca,NaN


In [ ]:
import pandas as pd

df = df[['text_spanish', 'target_column']].dropna()
df['target_column'] = df['target_column'].astype(int)
df = df.rename(columns={'text_spanish': 'text', 'target_column': 'label'})

# Subsample 20% for faster training
df = df.sample(frac=0.2, random_state=42).reset_index(drop=True)

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, stratify=df['label'], random_state=42
)

tokenizer = AutoTokenizer.from_pretrained("bertin-project/bertin-roberta-base-spanish")

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)

train_dataset = Dataset.from_dict({**train_encodings, "label": list(train_labels)})
eval_dataset = Dataset.from_dict({**val_encodings, "label": list(val_labels)})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "precision": precision_score(labels, preds, average="weighted", zero_division=0),
        "recall": recall_score(labels, preds, average="weighted", zero_division=0),
        "f1": f1_score(labels, preds, average="weighted")
    }

def model_init():
    from transformers import AutoModelForSequenceClassification
    return AutoModelForSequenceClassification.from_pretrained(
        "bertin-project/bertin-roberta-base-spanish", num_labels=2
    )

## BERTIN Grid Search: Actual Hyperparameter Ranges Used

| Parameter                     | Values Tried               |
|------------------------------|----------------------------|
| `learning_rate`              | `[2e-5, 3e-5, 5e-5]`       |
| `per_device_train_batch_size`| `[16]`                     |
| `num_train_epochs`           | `[3, 4]`                   |
| `weight_decay`               | `[0.0, 0.01]`              |

In [ ]:
import optuna
from transformers import Trainer, TrainingArguments

def hp_space(trial):
    return {
        "learning_rate": trial.suggest_categorical("learning_rate", [2e-5, 3e-5, 5e-5]),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16]),
        "num_train_epochs": trial.suggest_categorical("num_train_epochs", [3, 4]),
        "weight_decay": trial.suggest_categorical("weight_decay", [0.0, 0.01])
    }

#Define training args
training_args = TrainingArguments(
    output_dir="./bertin-results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs-bertin",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)

trainer = Trainer(
    args=training_args,
    model_init=model_init,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Run Optuna search for 4 trials
best_run = trainer.hyperparameter_search(
    direction="maximize",
    hp_space=hp_space,
    n_trials=4
)

import json
import os

os.makedirs("/content/drive/MyDrive/BERTIN", exist_ok=True)

with open("/content/drive/MyDrive/BERTIN/bertin_gridsearch_best_results_binary.json", "w") as f:
    json.dump({
        "best_score": best_run.objective,
        "best_hyperparameters": best_run.hyperparameters
    }, f, indent=4)

with open("/content/drive/MyDrive/BERTIN/bertin_gridsearch_all_trials_binary.txt", "w") as f:
    for trial in best_run.trials:
        f.write(f"Trial {trial.number} | F1: {trial.value:.4f} | Params: {trial.params}\n")

/tmp/ipython-input-14-3761169485.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at bertin-project/bertin-roberta-base-spanish and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2025-07-04 06:46:04,104] A new study created in memory with name: no-name-3c5a07e4-0bef-495f-9782-5d83afd5cd6e
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at bertin-project/bertin-roberta-base-spanish and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<IPython.core.display.Javascript object>

...

[I 2025-07-04 08:01:34,915] Trial 2 finished with value: 2.99807759122866 and parameters: {'learning_rate': 5e-05, 'per_device_train_batch_size': 16, 'num_train_epochs': 3, 'weight_decay': 0.0}. Best is trial 1 with value: 3.0757870863994183.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at bertin-project/bertin-roberta-base-spanish and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


eval/accuracy,▁█▇
eval/f1,▁██
eval/loss,█▆▁
eval/precision,▁██
eval/recall,▁█▇
eval/runtime,█▁▅
eval/samples_per_second,▁█▄
eval/steps_per_second,▁█▅
train/epoch,▁▁▅▅██
train/global_step,▁▁▄▅██
train/grad_norm,▁█


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.565291,0.715896,0.512507,0.715896,0.597364
2,0.585300,0.554233,0.760428,0.748101,0.760428,0.751255
3,0.516200,0.523947,0.753664,0.743648,0.753664,0.747101


[I 2025-07-04 08:23:53,630] Trial 3 finished with value: 2.99807759122866 and parameters: {'learning_rate': 5e-05, 'per_device_train_batch_size': 16, 'num_train_epochs': 3, 'weight_decay': 0.0}. Best is trial 1 with value: 3.0757870863994183.


AttributeError: 'BestRun' object has no attribute 'trials'

In [ ]:
import json

# Save best trial only
best_path = "/content/drive/MyDrive/BERTIN/bertin_gridsearch_best_results_binary.json"

with open(best_path, "w") as f:
    json.dump({
        "best_score": best_run.objective,
        "best_hyperparameters": best_run.hyperparameters
    }, f, indent=4)

print("✅ Best run saved to Google Drive.")

✅ Best run saved to Google Drive.


In [ ]:
print("🔍 Best Trial Hyperparameters:")
for param, value in best_run.hyperparameters.items():
    print(f"{param}: {value}")

print(f"\n🏆 Best F1 Score: {best_run.objective:.4f}")

🔍 Best Trial Hyperparameters:
learning_rate: 5e-05
per_device_train_batch_size: 16
num_train_epochs: 4
weight_decay: 0.01

🏆 Best F1 Score: 3.0758
